In [87]:
import scipy.io
import pandas as pd
import numpy as np
import sys
import os
import pandas as pd
import scipy.io
def matFiles(df='path'):
    #Consistent parameters to use for editing datasets
    nrois=333
    #Load FC file
    fileFC=scipy.io.loadmat(df)
    #Convert to numpy array
    fileFC=np.array(fileFC['parcel_corrmat'])
    #Replace nans and infs with zero
    fileFC=np.nan_to_num(fileFC)
    nsess=fileFC.shape[2]
    #Index upper triangle of matrix
    mask=np.triu_indices(nrois,1)
    ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
    count=0
    #Loop through all 10 days to reshape correlations into linear form
    for sess in range(nsess):
        tmp=fileFC[:,:,sess]
        ds[count]=tmp[mask]
        count=count+1
    return ds

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
import numpy as np
import os
import sys
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
import itertools
#import other python scripts for further anlaysis
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/ESA/'
clf=LinearSVC()

subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#load all subs for rest and mem task
allTask = np.empty((0,55278))
allRest = np.empty((0,55278))
for sub in subList:
    taskFC=matFiles(dataDir+'mem/'+sub+'_parcel_corrmat.mat')

    restFC=matFiles(dataDir+'rest/'+sub+'_parcel_corrmat.mat')

    allTask = np.append(allTask, taskFC, axis=0)
    allRest = np.append(allRest, restFC, axis=0)
    

taskSize=allTask.shape[0]
restSize=allRest.shape[0]

t = np.ones(taskSize, dtype = int)
r=np.zeros(restSize, dtype=int)

x=np.append(allTask, allRest, axis=0)
y=np.append(t,r, axis=0)

#test set
testX = np.empty((0,55278))

for sub in subList:
    test=matFiles(dataDir+'mixed/'+sub+'_parcel_corrmat.mat')
    testX = np.append(testX, test, axis=0)
testSize=testX.shape[0]
testY = np.ones(testSize, dtype = int)

kf = KFold(n_splits=10)
df=pd.DataFrame()
acc_score=[]
for train_index, test_index in kf.split(x):
    xSet=x[train_index]
    ySet=y[train_index]   
    clf.fit(xSet, ySet)
    clf.predict(testX)
    acc=clf.score(testX, testY)
    acc_score.append(acc)
df['scores']=acc_score

In [89]:
df

,scores
0,0.5375
1,0.5625
2,0.5750
3,0.5625
4,0.5500
5,0.6625
6,0.7375
7,0.6875
8,0.6750
9,0.6875
